# VendorClose AI - Smart End-of-Day Fruit Scanner
## Machine Learning Pipeline: Training, Evaluation, and Testing

This notebook demonstrates:
1. Data Acquisition and Preprocessing
2. Model Creation with Transfer Learning
3. Model Training with Optimization Techniques
4. Comprehensive Model Evaluation (4+ metrics)
5. Model Testing and Predictions


## 1. Setup and Imports


In [1]:
import sys
import os
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import pandas as pd

# Import our custom modules
from src.preprocessing import ImagePreprocessor, create_medium_class_from_dataset
from src.model import FruitQualityClassifier
from src.prediction import FruitPredictor

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")


TensorFlow version: 2.20.0
GPU Available: []


## 2. Data Acquisition


In [2]:
# Data directory structure
data_dir = Path('../data')
train_dir = data_dir / 'train'
test_dir = data_dir / 'test'

print("📥 Data Acquisition")
print("=" * 50)
print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")

# Check if directories exist
if train_dir.exists():
    print("\n✅ Training data directory found")
    # Count images in each class
    for class_dir in train_dir.iterdir():
        if class_dir.is_dir():
            img_count = len(list(class_dir.glob('*.jpg'))) + len(list(class_dir.glob('*.png')))
            print(f"  - {class_dir.name}: {img_count} images")
else:
    print("\n⚠️  Training data directory not found")
    print("Please download the dataset from:")
    print("https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification")


📥 Data Acquisition
Train directory: ..\data\train
Test directory: ..\data\test

✅ Training data directory found
  - freshapples: 2424 images
  - freshbanana: 2468 images
  - freshbittergroud: 327 images
  - freshcapsicum: 990 images
  - freshcucumber: 496 images
  - freshokra: 635 images
  - freshoranges: 1466 images
  - freshpotato: 536 images
  - freshtomato: 1858 images
  - rottenapples: 3248 images
  - rottenbanana: 2932 images
  - rottenbittergroud: 357 images
  - rottencapsicum: 901 images
  - rottencucumber: 421 images
  - rottenokra: 338 images
  - rottenoranges: 1595 images
  - rottenpotato: 802 images
  - rottentomato: 1825 images


## 3. Data Preprocessing


In [3]:
# Create medium class if needed
if train_dir.exists():
    create_medium_class_from_dataset('../data')

# Initialize preprocessor
preprocessor = ImagePreprocessor(img_size=(224, 224), batch_size=32)

# Create data generators with augmentation
print("\n📊 Data Preprocessing")
print("=" * 50)

if train_dir.exists():
    train_gen, val_gen, test_gen = preprocessor.create_data_generators(
        train_dir=str(train_dir),
        val_dir=str(train_dir),  # Validation split from training
        test_dir=str(test_dir) if test_dir.exists() else None
    )
    
    print(f"\n✅ Data generators created")
    print(f"Classes: {train_gen.class_indices}")
    print(f"Training samples: {train_gen.samples}")
    print(f"Validation samples: {val_gen.samples}")
    if test_gen:
        print(f"Test samples: {test_gen.samples}")
else:
    print("⚠️  Cannot create data generators - data directory not found")


No medium class images created - ensure fresh/rotten classes exist

📊 Data Preprocessing
Found 18901 images belonging to 19 classes.
Found 4718 images belonging to 19 classes.
Found 6738 images belonging to 14 classes.

✅ Data generators created
Classes: {'freshapples': 0, 'freshbanana': 1, 'freshbittergroud': 2, 'freshcapsicum': 3, 'freshcucumber': 4, 'freshokra': 5, 'freshoranges': 6, 'freshpotato': 7, 'freshtomato': 8, 'medium': 9, 'rottenapples': 10, 'rottenbanana': 11, 'rottenbittergroud': 12, 'rottencapsicum': 13, 'rottencucumber': 14, 'rottenokra': 15, 'rottenoranges': 16, 'rottenpotato': 17, 'rottentomato': 18}
Training samples: 18901
Validation samples: 4718
Test samples: 6738


## 4. Model Creation


In [ ]:
print("\n🏗️  Model Creation")
print("=" * 50)

# Get number of classes from data generator
if 'train_gen' in locals():
    num_classes = len(train_gen.class_indices)
    print(f"Detected {num_classes} classes from data")
    print(f"Classes: {list(train_gen.class_indices.keys())}")
else:
    num_classes = 3  # Default fallback
    print(f"Warning: Using default {num_classes} classes (data generator not found)")

# Initialize classifier with correct number of classes
classifier = FruitQualityClassifier(
    num_classes=num_classes,  # Use detected number of classes
    img_size=(224, 224),
    learning_rate=0.001
)

# Build model with transfer learning (MobileNetV2)
model = classifier.build_model(use_pretrained=True)

# Compile model with optimizer
classifier.compile_model(optimizer_name='adam')

# Display model summary
print("\n📋 Model Architecture:")
model.summary()

print("\n✅ Model created with:")
print("  - Transfer Learning: MobileNetV2 (pretrained on ImageNet)")
print("  - Regularization: Dropout, L2, Batch Normalization")
print("  - Data Augmentation: Rotation, Zoom, Shift, Flip")
print("  - Optimizer: Adam")
print("  - Metrics: Accuracy, Precision, Recall, AUC")



🏗️  Model Creation
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 8s 1us/step

📋 Model Architecture:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,427,971 (9.26 MB)

 Trainable params: 167,171 (653.01 KB)

 Non-trainable params: 2,260,800 (8.62 MB)


✅ Model created with:
  - Transfer Learning: MobileNetV2 (pretrained on ImageNet)
  - Regularization: Dropout, L2, Batch Normalization
  - Data Augmentation: Rotation, Zoom, Shift, Flip
  - Optimizer: Adam
  - Metrics: Accuracy, Precision, Recall, AUC


## 5. Model Training


In [ ]:
print("\n🚀 Model Training")
print("=" * 50)

if train_dir.exists() and 'train_gen' in locals():
    # Ensure model output matches data generator classes to avoid shape mismatch
    num_classes = len(train_gen.class_indices)
    current_output = None
    if hasattr(classifier, 'model') and classifier.model is not None:
        try:
            current_output = classifier.model.output_shape[-1]
        except Exception:
            current_output = None

    if current_output != num_classes:
        print(f"⚠️  Mismatch detected: train generator has {num_classes} classes "
              f"but model outputs {current_output}. Rebuilding model to match classes...")
        # Rebuild classifier with correct number of classes
        # Preserve learning rate if available, otherwise default to 0.001
        lr = getattr(classifier, 'learning_rate', 0.001)
        classifier = FruitQualityClassifier(
            num_classes=num_classes,
            img_size=(224, 224),
            learning_rate=lr
        )
        # Build and compile new model
        model = classifier.build_model(use_pretrained=True)
        classifier.compile_model(optimizer_name='adam')
    else:
        # Make sure `model` variable references the classifier's model
        model = getattr(classifier, 'model', None)

    # Train the model
    # Limit the maximum epochs to 10 so training doesn't run unnecessarily long.
    # EarlyStopping (in `src/model.py`) will restore the best weights and stop earlier if no improvement.
    train_epochs = 10  # set a conservative max; adjust if you want longer training
    history = classifier.train(
        train_generator=train_gen,
        val_generator=val_gen,
        epochs=train_epochs,  # EarlyStopping may stop earlier
        batch_size=32
    )
    
    # Save the trained model
    classifier.save_model('../models/fruit_classifier.h5')
    
    # Plot training history
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy
    axes[0, 0].plot(history.history.get('accuracy', []), label='Training Accuracy')
    axes[0, 0].plot(history.history.get('val_accuracy', []), label='Validation Accuracy')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # Loss
    axes[0, 1].plot(history.history.get('loss', []), label='Training Loss')
    axes[0, 1].plot(history.history.get('val_loss', []), label='Validation Loss')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Precision
    axes[1, 0].plot(history.history.get('precision', []), label='Training Precision')
    axes[1, 0].plot(history.history.get('val_precision', []), label='Validation Precision')
    axes[1, 0].set_title('Model Precision')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # Recall
    axes[1, 1].plot(history.history.get('recall', []), label='Training Recall')
    axes[1, 1].plot(history.history.get('val_recall', []), label='Validation Recall')
    axes[1, 1].set_title('Model Recall')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.savefig('../logs/training_history.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Training completed!")
else:
    print("⚠️  Cannot train - data generators not available")



🚀 Model Training
Epoch 1/50
  4/591 ━━━━━━━━━━━━━━━━━━━━ 12:54 1s/step - accuracy: 0.7487 - auc: 0.9736 - loss: 3.4554 - precision: 0.9126 - recall: 0.4310

KeyboardInterrupt: 

In [ ]:
print("\n📊 Model Evaluation")
print("=" * 50)

if 'val_gen' in locals() and classifier.model is not None:
    # Evaluate on validation set
    eval_results = classifier.model.evaluate(val_gen, verbose=1)
    
    print("\n📈 Evaluation Metrics:")
    print(f"  Loss: {eval_results[0]:.4f}")
    print(f"  Accuracy: {eval_results[1]:.4f} ({eval_results[1]*100:.2f}%)")
    print(f"  Precision: {eval_results[2]:.4f} ({eval_results[2]*100:.2f}%)")
    print(f"  Recall: {eval_results[3]:.4f} ({eval_results[3]*100:.2f}%)")
    print(f"  AUC: {eval_results[4]:.4f}")
    
    # Get predictions for detailed metrics
    val_gen.reset()
    y_pred = classifier.model.predict(val_gen, verbose=1)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = val_gen.classes
    
    # Calculate F1 Score
    f1 = f1_score(y_true, y_pred_classes, average='weighted')
    precision = precision_score(y_true, y_pred_classes, average='weighted')
    recall = recall_score(y_true, y_pred_classes, average='weighted')
    
    print("\n📊 Detailed Metrics:")
    print(f"  Weighted F1 Score: {f1:.4f} ({f1*100:.2f}%)")
    print(f"  Weighted Precision: {precision:.4f} ({precision*100:.2f}%)")
    print(f"  Weighted Recall: {recall:.4f} ({recall*100:.2f}%)")
    
    # Classification Report
    class_names = list(val_gen.class_indices.keys())
    print("\n📋 Classification Report:")
    print(classification_report(y_true, y_pred_classes, target_names=class_names))
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig('../logs/confusion_matrix.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Metrics Summary DataFrame
    metrics_df = pd.DataFrame({
        'Metric': ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC'],
        'Value': [eval_results[0], eval_results[1], precision, recall, f1, eval_results[4]]
    })
    print("\n📊 Metrics Summary:")
    print(metrics_df.to_string(index=False))
    
    print("\n✅ Evaluation completed with 6 metrics:")
    print("  1. Loss (Categorical Crossentropy)")
    print("  2. Accuracy")
    print("  3. Precision (Weighted)")
    print("  4. Recall (Weighted)")
    print("  5. F1 Score (Weighted)")
    print("  6. AUC (Area Under Curve)")
else:
    print("⚠️  Cannot evaluate - model or validation generator not available")


In [ ]:
print("\n🧪 Model Testing")
print("=" * 50)

# Load predictor
predictor = FruitPredictor(model_path='../models/fruit_classifier.h5')

try:
    predictor.load_model()
    
    # Test on a few sample images
    if test_dir.exists():
        # Get sample images from each class
        test_images = []
        for class_dir in test_dir.iterdir():
            if class_dir.is_dir():
                images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.png'))
                if images:
                    test_images.append(images[0])
        
        if test_images:
            print(f"\n📸 Testing on {len(test_images)} sample images:")
            
            fig, axes = plt.subplots(1, len(test_images), figsize=(15, 5))
            if len(test_images) == 1:
                axes = [axes]
            
            for i, img_path in enumerate(test_images):
                result = predictor.predict_single(img_path)
                
                # Display image
                img = plt.imread(img_path)
                axes[i].imshow(img)
                axes[i].set_title(f"{result['class']}\nConfidence: {result['confidence']:.2%}\n{result['action']}")
                axes[i].axis('off')
                
                print(f"\n  Image: {img_path.name}")
                print(f"    Prediction: {result['class']}")
                print(f"    Confidence: {result['confidence']:.2%}")
                print(f"    Action: {result['action']}")
            
            plt.tight_layout()
            plt.savefig('../logs/test_predictions.png', dpi=150, bbox_inches='tight')
            plt.show()
            
            print("\n✅ Testing completed!")
        else:
            print("⚠️  No test images found")
    else:
        print("⚠️  Test directory not found")
        
except Exception as e:
    print(f"⚠️  Error during testing: {e}")
    print("Make sure the model has been trained first!")


## 8. Feature Visualization and Interpretation


In [ ]:
print("\n📊 Feature Visualization")
print("=" * 50)

if 'val_gen' in locals() and classifier.model is not None:
    # Get class distribution
    class_counts = {}
    for class_name in val_gen.class_indices.keys():
        class_dir = train_dir / class_name
        if class_dir.exists():
            count = len(list(class_dir.glob('*.jpg'))) + len(list(class_dir.glob('*.png')))
            class_counts[class_name] = count
    
    # Create visualizations
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 1. Class Distribution
    axes[0].bar(class_counts.keys(), class_counts.values(), color=['green', 'orange', 'red'])
    axes[0].set_title('Training Data: Class Distribution', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Fruit Quality Class', fontsize=12)
    axes[0].set_ylabel('Number of Images', fontsize=12)
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(axis='y', alpha=0.3)
    
    # 2. Prediction Confidence Distribution
    if 'y_pred' in locals():
        max_probs = np.max(y_pred, axis=1)
        axes[1].hist(max_probs, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
        axes[1].set_title('Prediction Confidence Distribution', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Confidence Score', fontsize=12)
        axes[1].set_ylabel('Frequency', fontsize=12)
        axes[1].axvline(np.mean(max_probs), color='red', linestyle='--', 
                       label=f'Mean: {np.mean(max_probs):.3f}')
        axes[1].legend()
        axes[1].grid(axis='y', alpha=0.3)
    
    # 3. Per-Class Accuracy
    if 'cm' in locals():
        class_accuracies = cm.diagonal() / cm.sum(axis=1)
        axes[2].bar(range(len(class_names)), class_accuracies, 
                   color=['green', 'orange', 'red'])
        axes[2].set_title('Per-Class Accuracy', fontsize=14, fontweight='bold')
        axes[2].set_xlabel('Fruit Quality Class', fontsize=12)
        axes[2].set_ylabel('Accuracy', fontsize=12)
        axes[2].set_xticks(range(len(class_names)))
        axes[2].set_xticklabels(class_names, rotation=45)
        axes[2].set_ylim([0, 1])
        axes[2].grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for i, acc in enumerate(class_accuracies):
            axes[2].text(i, acc + 0.02, f'{acc:.2%}', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../logs/feature_visualizations.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n📖 Feature Interpretations:")
    print("\n1. Class Distribution:")
    print("   - Shows the balance of training data across quality classes")
    print("   - Balanced datasets lead to better model performance")
    
    print("\n2. Prediction Confidence:")
    print("   - Higher confidence scores indicate model certainty")
    print("   - Distribution helps identify ambiguous cases")
    
    print("\n3. Per-Class Accuracy:")
    print("   - Reveals which quality classes are easier/harder to predict")
    print("   - Helps identify areas for model improvement")
    
    print("\n✅ Feature visualization completed!")
else:
    print("⚠️  Cannot create visualizations - model or data not available")
